# Step 1

In [1]:
# Write a program that grabs sector change data from google finance at the end of each day
# Start collecting daily data by running this function after the market close every day
# And save the JSON object somewhere - in a file perhaps

In [2]:
def google_finance_sectors():

    import requests
    import bs4
    import json

    response = requests.get('https://www.google.com/finance')
    soup = bs4.BeautifulSoup(response.content, 'lxml')
    sector_dict = {}
    raw_data = soup.find('table', width="100%", cellpadding=0, cellspacing=0)
    # sector_change
    sector_change = []
    for _ in raw_data.find_all('span'):
        sector_change.append(_.get_text().rstrip('%').lstrip('+'))
    # url of the sector and other imformation
    i = 0
    for sector in raw_data.find_all('a'):
        url = 'https://www.google.com' + sector.get('href')
        sector_response = requests.get(url)
        sector_soup = bs4.BeautifulSoup(sector_response.content, 'lxml')
        # sector_name
        sector_name = sector_soup.find('h3').get_text().strip()
        sector_dict[sector_name] = {}
        # sector_change
        sector_dict[sector_name]['sector_change'] = float(sector_change[i])
        i += 1
        #top_gainer & top_loser & top_mostactive
        topmovers_data = sector_soup.find('table', {"class": "topmovers"})
        topmovers_data_change = []
        for _ in topmovers_data.find_all('span'):
            if '%' in _.get_text():
                topmovers_data_change.append(
                    _.get_text().strip('()').rstrip('%').lstrip('+'))
        topmovers_data_name = []
        for _ in topmovers_data.find_all('a'):
            topmovers_data_name.append(_.get_text())
        sector_dict[sector_name]['top_gainer'] = {}
        sector_dict[sector_name]['top_loser'] = {}
        sector_dict[sector_name]['top_mostactive'] = {}
        if len(topmovers_data_change) == 15:
            gainer_stock = topmovers_data_name[0]
            gainer_stock_change = topmovers_data_change[0]
            loser_stock = topmovers_data_name[10]
            loser_stock_change = topmovers_data_change[5]
            active_stock = topmovers_data_name[20]
            active_stock_change = topmovers_data_change[10]
            sector_dict[sector_name]['top_gainer'][
                'stock_change'] = float(gainer_stock_change)
            sector_dict[sector_name]['top_loser'][
                'stock_change'] = float(loser_stock_change)
            sector_dict[sector_name]['top_mostactive'][
                'stock_change'] = float(active_stock_change)
        elif 'Gainers' in topmovers_data.find('b').get_text():
            gainer_stock = topmovers_data_name[0]
            gainer_stock_change = topmovers_data_change[0]
            loser_stock = ''
            loser_stock_change = None
            active_stock = topmovers_data_name[10]
            active_stock_change = topmovers_data_change[5]
            sector_dict[sector_name]['top_gainer'][
                'stock_change'] = float(gainer_stock_change)
            sector_dict[sector_name]['top_loser'][
                'stock_change'] = loser_stock_change
            sector_dict[sector_name]['top_mostactive'][
                'stock_change'] = float(active_stock_change)
        else:
            gainer_stock = ''
            gainer_stock_change = None
            loser_stock = topmovers_data_name[0]
            loser_stock_change = topmovers_data_change[0]
            active_stock = topmovers_data_name[10]
            active_stock_change = topmovers_data_change[5]
            sector_dict[sector_name]['top_gainer'][
                'stock_change'] = gainer_stock_change
            sector_dict[sector_name]['top_loser'][
                'stock_change'] = float(loser_stock_change)
            sector_dict[sector_name]['top_mostactive'][
                'stock_change'] = float(active_stock_change)

        sector_dict[sector_name]['top_gainer']['stock'] = gainer_stock
        sector_dict[sector_name]['top_loser']['stock'] = loser_stock
        sector_dict[sector_name]['top_mostactive']['stock'] = active_stock

    return json.dumps(sector_dict)

In [3]:
google_finance_sectors()

'{"Financials": {"top_mostactive": {"stock_change": 0.73, "stock": "SPDR S&P 500 ETF Trust"}, "top_loser": {"stock_change": -9.01, "stock": "Banco Santander Brasil SA"}, "sector_change": 0.88, "top_gainer": {"stock_change": 7.6, "stock": "German American Bancorp"}}, "Healthcare": {"top_mostactive": {"stock_change": 0.45, "stock": "UnitedHealth Group Inc"}, "top_loser": {"stock_change": -4.81, "stock": "CTI BioPharma Corp"}, "sector_change": -0.1, "top_gainer": {"stock_change": 11.29, "stock": "Omeros Corporation"}}, "Telecommunications Services": {"top_mostactive": {"stock_change": 0.17, "stock": "AT&T Inc."}, "top_loser": {"stock_change": -4.06, "stock": "Partner Communications Co"}, "sector_change": -0.03, "top_gainer": {"stock_change": 1.77, "stock": "Mobil\'nye Telesistemy PAO"}}, "Industrials": {"top_mostactive": {"stock_change": 0.61, "stock": "General Electric Company"}, "top_loser": {"stock_change": -5.72, "stock": "FactSet Research Systems"}, "sector_change": 0.83, "top_gainer

# Step 2

In [4]:
# Design a MongoDB collection that stores the data
# Only running this at the very begining
from pymongo import MongoClient
import datetime
import json

client = MongoClient()
sectorsCollection = client.google_finance.google_finance_sectors
#sectorsCollection.delete_many({})
date = str(datetime.date.today())
data = json.loads(google_finance_sectors())
document = {"date": date}
for key in data:
    document[key] = data[key]
# create when first insert
sectorsCollection.insert_one(document)

In [5]:
# And write a function that updates the collection as new data arrives
# You can run this function at any time you want


def update_collection():
    from pymongo import MongoClient
    import datetime
    import json

    client = MongoClient()
    sectorsCollection = client.google_finance.google_finance_sectors
    new_date = str(datetime.date.today())
    new_data = json.loads(google_finance_sectors())
    new_document = {"date": new_date}
    for key in new_data:
        new_document[key] = new_data[key]
    if not sectorsCollection.find_one({"date": new_date}):
        sectorsCollection.insert_one(new_document)

In [6]:
update_collection()
for _ in sectorsCollection.find():
    print(_)

{'date': '2017-03-28', 'Non-Cyclical Consumer Goods & Services': {'top_mostactive': {'stock_change': 0.3, 'stock': 'Procter & Gamble Co'}, 'sector_change': 0.52, 'top_loser': {'stock_change': -10.87, 'stock': 'StoneMor Partners L.P.'}, 'top_gainer': {'stock_change': None, 'stock': ''}}, 'Energy': {'top_mostactive': {'stock_change': 0.73, 'stock': 'Exxon Mobil Corporation'}, 'sector_change': 1.03, 'top_loser': {'stock_change': -6.22, 'stock': 'Gulf Island Fabrication'}, 'top_gainer': {'stock_change': 9.11, 'stock': 'Broadwind Energy Inc.'}}, '_id': ObjectId('58daf32a91c07638c3eb1642'), 'Utilities': {'top_mostactive': {'stock_change': -0.41, 'stock': 'Southern Co'}, 'sector_change': 0.05, 'top_loser': {'stock_change': -2.41, 'stock': 'Cia Energetica de Minas'}, 'top_gainer': {'stock_change': 5.69, 'stock': 'MGE Energy, Inc.'}}, 'Financials': {'top_mostactive': {'stock_change': 0.73, 'stock': 'SPDR S&P 500 ETF Trust'}, 'sector_change': 0.88, 'top_loser': {'stock_change': -9.01, 'stock': '

# Step 3

In [7]:
# Design an interface object for your collection
# Answer the following nine questions

In [8]:
class google_finance_sectors_interface():

    def __init__(self):
        from pymongo import MongoClient
        self._collection = MongoClient().google_finance.google_finance_sectors

    # 1.Which sector has had the greatest move on a given date
    def sector_greatest_move_given_date(self, given_date):
        data = self._collection.find_one({"date": given_date})
        max_change = 0
        for key in data:
            if key == "date":
                pass
            elif key == '_id':
                pass
            else:
                if max_change < data[key]['sector_change']:
                    max_change = data[key]['sector_change']
                    max_sector = key
        return max_sector, max_change

    # 2.Which sector has had the greatest move ever
    def sector_greatest_move_ever(self):
        max_change_ever = 0
        for doc in self._collection.find():
            date = doc["date"]
            max_sector, max_change = self.sector_greatest_move_given_date(date)
            if max_change_ever < max_change:
                max_change_ever = max_change
                max_sector_ever = max_sector
                max_date_ever = date
        return max_sector_ever, max_change_ever, max_date_ever

    # 3.Which stock has had the greatest move on a given date
    def stock_greatest_move_given_date(self, given_date):
        data = self._collection.find_one({"date": given_date})
        max_change = 0
        for key in data:
            if key == "date":
                pass
            elif key == '_id':
                pass
            else:
                if data[key]['top_gainer']['stock_change']:
                    if max_change < data[key]['top_gainer']['stock_change']:
                        max_change = data[key]['top_gainer']['stock_change']
                        max_stock = data[key]['top_gainer']['stock']
        return max_stock, max_change
    
    # 4.Which stock has had the greatest move ever
    def stock_greatest_move_ever(self):
        max_change_ever = 0
        for doc in self._collection.find():
            date = doc["date"]
            max_stock, max_change = self.stock_greatest_move_given_date(date)
            if max_change_ever < max_change:
                max_change_ever = max_change
                max_stock_ever = max_stock
                max_date_ever = date
        return max_stock_ever, max_change_ever, max_date_ever

    # 5.Return a list of (date, change) tuples for a given sector
    def sector_history(self, given_sector):
        sector_list = []
        for doc in self._collection.find():
            date = doc["date"]
            change = doc[given_sector]['sector_change']
            sector_list.append((date, change))
        return sector_list

    # 6.Which stock has had the greatest loss on a given date
    def stock_greatest_loss_given_date(self, given_date):
        data = self._collection.find_one({"date": given_date})
        max_change = 0
        for key in data:
            if key == "date":
                pass
            elif key == '_id':
                pass
            else:
                if data[key]['top_loser']['stock_change']:
                    if max_change > data[key]['top_loser']['stock_change']:
                        max_change = data[key]['top_loser']['stock_change']
                        max_stock = data[key]['top_loser']['stock']
        return max_stock, max_change

    # 7.Which stock has had the greatest loss ever
    def stock_greatest_loss_ever(self):
        max_change_ever = 0
        for doc in self._collection.find():
            date = doc["date"]
            max_stock, max_change = self.stock_greatest_loss_given_date(date)
            if max_change_ever > max_change:
                max_change_ever = max_change
                max_stock_ever = max_stock
                max_date_ever = date
        return max_stock_ever, max_change_ever, max_date_ever

    # 8.Which stock has had the greatest move on a given date
    def stock_mostactive_given_date(self, given_date):
        data = self._collection.find_one({"date": given_date})
        max_change = 0
        for key in data:
            if key == "date":
                pass
            elif key == '_id':
                pass
            else:
                if data[key]['top_mostactive']['stock_change']:
                    if max_change < data[key]['top_mostactive']['stock_change']:
                        max_change = data[key][
                            'top_mostactive']['stock_change']
                        max_stock = data[key]['top_mostactive']['stock']
        return max_stock, max_change

    # 9.Which stock has had the greatest move ever
    def stock_mostactive_ever(self):
        max_change_ever = 0
        for doc in self._collection.find():
            date = doc["date"]
            max_stock, max_change = self.stock_mostactive_given_date(date)
            if max_change_ever < max_change:
                max_change_ever = max_change
                max_stock_ever = max_stock
                max_date_ever = date
        return max_stock_ever, max_change_ever, max_date_ever

In [9]:
test = google_finance_sectors_interface()

In [10]:
test.sector_greatest_move_given_date('2017-03-28')

('Energy', 1.03)

In [11]:
test.sector_greatest_move_ever()

('Energy', 1.03, '2017-03-28')

In [12]:
test.stock_greatest_move_given_date('2017-03-28')

('LML Payment Systems, Inc.', 633.43)

In [13]:
test.stock_greatest_move_ever()

('LML Payment Systems, Inc.', 633.43, '2017-03-28')

In [14]:
test.sector_history('Financials')

[('2017-03-28', 0.88)]

In [15]:
test.stock_greatest_loss_given_date('2017-03-28')

('StoneMor Partners L.P.', -10.87)

In [16]:
test.stock_greatest_loss_ever()

('StoneMor Partners L.P.', -10.87, '2017-03-28')

In [17]:
test.stock_mostactive_given_date('2017-03-28')

('Tesla Inc', 2.68)

In [18]:
test.stock_mostactive_ever()

('Tesla Inc', 2.68, '2017-03-28')